In [ ]:
import torch
import torchvision
import torch.nn as nn

In [ ]:
#Lenet architecture
1x32x32 - input
k = 5x5, s = 1, p = 0
avg pool, s = 2, p =0
k = 5x5, s=1, p=0
avg poo; s=2, p=0
conv 5x5 
120 x linear 84x Linear 10

In [ ]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.relu = nn.ReLU()
        self.pool = nn.AvgPool2d(kernel_size=(2,2), stride=(2,2))
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=(5,5), stride=(1,1), padding=(0,0))
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=(5,5), stride=(1,1), padding=(0,0))
        self.conv3 = nn.Conv2d(in_channels=16, out_channels=120, kernel_size=(5,5), stride=(1,1), padding=(0,0))
        self.linear1 = nn.Linear(120, 84)
        self.linear2 = nn.Linear(84, 10)
    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.pool(x)
        x = self.relu(self.conv2(x))
        x = self.pool(x)
        x = self.relu(self.conv3(x))
        x = x.reshape(x.shape[0], -1)
        x = self.relu(self.linear1(x))
        x = self.linear2(x)
        return x


In [ ]:
x = torch.randn(64, 1, 32, 32)
model = LeNet()
print(model(x).shape)

torch.Size([64, 10])


# Inception **Net** **bold text**

In [ ]:
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, **kwargs):
        super(ConvBlock, self).__init__()
        self.relu = nn.ReLU()
        self.conv = nn.Conv2d(in_channels, out_channels, **kwargs)
        self.batchnorm = nn.BatchNorm2d(out_channels)

    def forward(self, x):
        return self.relu(self.batchnorm(self.conv(x)))


class InceptionBlock(nn.Module):
    def __init__(self, in_channels, out_1x1, red_3x3, out_3x3, red_5x5, out_5x5, out_1x1pool):
        super(InceptionBlock, self).__init__()

        self.branch1 = ConvBlock(in_channels, out_1x1, kernel_size = (1,1))

        self.branch2 = nn.Sequential(ConvBlock(in_channels, red_3x3, kernel_size = (1,1)),
                                     ConvBlock(red_3x3, out_3x3, kernel_size = (3,3), padding = (1,1)))
        
        self.branch3 = nn.Sequential(ConvBlock(in_channels, red_5x5, kernel_size = (1,1)),
                                     ConvBlock(red_5x5, out_5x5, kernel_size = (5,5), padding = (2,2)))
        
        self.branch4 = nn.Sequential(nn.MaxPool2d(kernel_size = (3,3), padding = (1,1), stride = (1,1)),
                                     ConvBlock(in_channels, out_1x1pool, kernel_size = (1,1)))


    def forward(self, x):
        return torch.cat([self.branch1(x), self.branch2(x), self.branch3(x), self.branch4(x)], 1) #(N, c, h, w) we are adding all the filters


class GoogleNet(nn.Module):
    def __init__(self, num_classes = 1000):
        super(GoogleNet, self).__init__()
        self.conv1 = ConvBlock(in_channels = 3, out_channels = 64, kernel_size=(7,7), padding = (3,3), stride = (2,2))
        self.maxpool1 = nn.MaxPool2d(kernel_size=(3,3), stride = (2,2), padding =(1,1))
        self.conv2 = ConvBlock(64, 192, kernel_size = (3, 3), stride = (1, 1), padding =(1,1))
        self.maxpool2 = nn.MaxPool2d(kernel_size = (3, 3), stride = (2, 2), padding = (1,1))
        self.inception1a = InceptionBlock(192, 64, 96, 128, 16, 32, 32)
        self.inception1b = InceptionBlock(256, 128, 128, 192, 32, 96, 64)
        self.maxpool3 = nn.MaxPool2d(kernel_size = (3, 3), stride = (2, 2), padding = (1,1))


        self.inception2a = InceptionBlock(480, 192, 96, 208, 16, 48, 64)
        self.inception2b = InceptionBlock(512, 160, 112, 224, 24, 64, 64)
        self.inception2c = InceptionBlock(512, 128, 128, 256, 24, 64, 64)
        self.inception2d = InceptionBlock(512, 112, 144, 288, 32, 64, 64)
        self.inception2e = InceptionBlock(528, 256, 160, 320, 32, 128, 128)


        self.maxpool4 = nn.MaxPool2d(kernel_size = (3, 3), stride = (2, 2), padding = (1,1))

        self.inception3a = InceptionBlock(832, 256, 160, 320, 32, 128, 128)
        self.inception3b = InceptionBlock(832, 384, 192, 384, 48, 128, 128) 

        self.avgpool = nn.AvgPool2d(kernel_size=(7, 7), stride = (1,1))
        self.dropout = nn.Dropout(p=0.4)
        self.fc1 = nn.Linear(1024, 1000)
    def forward(self, x):
        x = self.conv1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.maxpool2(x)

        x = self.inception1a(x)
        x = self.inception1b(x)
        x = self.maxpool3(x)

        x = self.inception2a(x)
        x = self.inception2b(x)
        x = self.inception2c(x)
        x = self.inception2d(x)
        x = self.inception2e(x)
        x = self.maxpool4(x)

        x = self.inception3a(x)
        x = self.inception3b(x)

        x = self.avgpool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.dropout(x)
        x = self.fc1(x)

        return x

In [ ]:
x = torch.randn(3, 3, 224, 224)
model = GoogleNet()
print(model(x).shape)

torch.Size([3, 1000])


# ResNet

In [ ]:
import torch
import torch.nn


class block(nn.Module):
    def __init__(self, in_channels, out_channels, identity_downsample=None, stride = (1,1)):
        super(block, self):
        self.expansion = 4
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, padding=0)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size = 3, stride=stride, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.conv3 = nn.Conv2d(out_channels, out_channels*self.expansion, kernel_size=1, stride=1, padding = 0 )
        self.bn3 = nn.BatchNorm2d(out_channels*self.expansion)
        self.relu = nn.ReLU()
        self.identity_downsample = identity_downsample

    def forward(self, x):
        identity = x

        x = self.conv1(x)
        x = self.bn1(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.conv3(x)
        x = self.bn3(x)

        if self.identity_downsample is not None:
            identity = self.identity_downsample(identity)

        x += identity
        x = self.relu(x)

        return x




class ResNet(nn.Module):
    def __init__(self, block, layers, image_channel, num_classes):
        super(ResNet, self).__init()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(image_channels, 64, kernel_size = 7 , stride = 2, padding = 3)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size = 3, stride = 2, padding = 1)


        # Resnet Layers

    def _make_layer(self, block, num_residual_blocks, out_channels, stride):
        